In [ ]:
import torch
import random

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #GPU를 사용할 수 있으면 사용하고 아니면 CPU사용

In [ ]:
# parameters
# 하이퍼파라미터. 학습하는 것이 아니라 정해줘야하는 값
learning_rate = 0.001 
epochs = 15
batch_size = 100     #전체 학습 데이터를 100개씩 넣는다. 모든 샘플이 학습이 되면 1 epoch
drop_out = 0.3 

# **데이터 준비**

In [ ]:
# MNIST dataset
training_data = datasets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(), #transform은 이미지를 tensor에 맞게 조정하기 위하여 생성
                          download=True)

test_data = datasets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)                                    

# **신경망 구성**

In [ ]:
# MNIST 이미지크기: 28 * 28 = 784
linear1 = nn.Linear(784, 512, bias=True)    # input layer (in, out)
linear2 = nn.Linear(512, 512, bias=True)    # 1st hidden layer  
linear3 = nn.Linear(512, 512, bias=True)    # 2nd hidden layer
linear4 = nn.Linear(512, 10, bias = True)   # output layer

relu = nn.ReLU() #activation function으로 ReLU 설정
dropout = nn.Dropout(p=drop_out) #dropout 설정

In [ ]:
#model 생성
# 선형 모델만 쓰면 레이어가 얼마나 있던 입력과 출력의 레이러로만 표현이 가능하기 때문에 비선형 함수인 relu함수를 넣어준다.
model = nn.Sequential(linear1, relu, dropout, 
                      linear2, relu, dropout,
                      linear3, relu, dropout,
                      linear4)
model = model.cuda()

In [ ]:
#cost & loss 명시하기 & optimizer
# pytorch는 softmax가 포함이 되어 있기 때문에 생략
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #gradient descent할 때 adam 활용

# **학습**

In [ ]:
total_batch = len(train_dataloader)
model.train()    # Training 시에는 dropout이 있으므로, 이를 활성화시키기 위함 set the model to train mode (dropout=True)

for epoch in range(epochs):
    avg_cost = 0

    for X, Y in train_dataloader:
        
        X = X.view(-1, 28 * 28).to(device)    # 입력 이미지를 [batch_size × 784]의 크기로 reshape
        Y = Y.to(device)                      # 레이블은 원-핫 인코딩이 된 상태가 아니라 0 ~ 9의 정수.

        optimizer.zero_grad()                 # optimizer 변수에 포함시킨 매개 변수(weight)들의 기울기(Gradient)를 0으로 초기화
        hypothesis = model(X)                 # 가설 설정
        cost = criterion(hypothesis, Y)       # 비용 설정
        cost.backward()                       # 역전파
        optimizer.step()                      # 학습을 통해 계산한 weight, bias, gradient를 최적화함수(optimizer)에에 반영

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

# **테스트**

In [ ]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # 테스트 시에는 dropout 적용 안함. set the model to evaluation mode (dropout=False)  

    # Test the model using test sets
    X_test = test_data.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = test_data.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    # 테스트 데이터 랜덤 샘플링하여 결과 예측
    r = random.randint(0, len(test_data) - 1)
    X_single_data = test_data.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = test_data.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())